In [1]:
import quimb as qu
import joblib
import numpy as np

In [2]:
# define number of sites == number of qubits (each site is a 2-level system)
n = 4

In [3]:
def h_single_site(n, k):
    dims = (2,) * n
    def gen_term(i):
        return qu.ikron(qu.pauli(k), dims, [i])
    return sum(map(gen_term, range(0, n)))    

def h_two_sites(n, k):
    dims = (2,) * n
    def gen_term(i):
        return qu.ikron(qu.pauli(k), dims, [i, i+1])
    return sum(map(gen_term, range(0, n-1)))

def h_two_sites_j2(n, k):
    dims = (2,) * n
    def gen_term(i):
        return qu.ikron(qu.pauli(k), dims, [i, i+2])
    return sum(map(gen_term, range(0, n-2)))

# Jordan-Wigner transformation
def annihilation_operator(n, index):
    dims = (2,) * n
    def gen_term(i, k):
        return qu.ikron(qu.pauli(k), dims, [i])
    term = 1/2 * (gen_term(index, "X") + 1j * gen_term(index, "Y"))
    for i in range(index-1, -1, -1):
        term = term @ gen_term(i, "Z")
    return term

def creation_operator(n, index):
    return annihilation_operator(n, index).H

Label 0 (1D transversefield Ising model):
- $H = k_1 \sum_{n=1}^{N-1} Z_n Z_{n+1} + k_2 \sum_{n=1}^{N} X_n \quad (k_1=1, k_2=2)$

Label 1 (1D Heisenberg model):
- $H = k_1 \sum_{n=1}^{N-1} (X_n X_{n+1} +  Y_n Y_{n+1} + Z_n Z_{n+1}) + k_2 \sum_{n=1}^{N} Z_n \quad (k_1=1, k_2=2)$

Label 2 (Su-Schrieffer-Heeger model):
- $H = k_1 \sum_{n=1}^{N-1} (X_n X_{n+1} +  Y_n Y_{n+1} + Z_n Z_{n+1}) + k_2 \sum_{n=1}^{N-1} (-1)^{n-1} (X_n X_{n+1} +  Y_n Y_{n+1} + Z_n Z_{n+1}) \quad (k_1=1, k_2=\frac{3}{2})$

Label 3 (J1-J2 model):
- $H = k_1 \sum_{n=1}^{N-1} (X_n X_{n+1} +  Y_n Y_{n+1} + Z_n Z_{n+1}) + k_2 \sum_{n=1}^{N-2} (X_n X_{n+2} +  Y_n Y_{n+2} + Z_n Z_{n+2}) \quad (k_1=1, k_2=3)$

Label 4 (1D Hubbard model):
- $H = -k_1 \sum_{j=1}^{N/2-1} \sum_{^\sigma \in \{\uparrow, \downarrow\}} (a_{j, \sigma}^\dagger a_{j+1, \sigma} + \mathrm{H.c.}) +
       k_2 \sum_{j=1}^{N/2} (a_{j, \uparrow}^\dagger a_{j, \uparrow} - \frac{1}{2}) (a_{j, \downarrow}^\dagger a_{j, \downarrow} - \frac{1}{2}) \quad (k_1=1, k_2=1)$  
$a_{j, \uparrow} = \frac{1}{2} (X_{2j} + i Y_{2j}) \prod_{k=2j-1}^{0} Z_k,  \quad
 a_{j, \downarrow} = \frac{1}{2} (X_{2j+1} + i Y_{2j+1}) \prod_{k=2j}^{0} Z_k$

Label 5 (2D Hubbard model):
- $H = -k_1 \sum_{^\sigma \in \{\uparrow, \downarrow\}} (\sum_{j_x=1}^{N/4-1} \sum_{j_y=1}^{2} a_{j_x, j_y, \sigma}^\dagger a_{j_x+1, j_y, \sigma} + 
                                                         \sum_{j_x=1}^{N/4} a_{j_x, 1, \sigma}^\dagger a_{j_x, 2, \sigma} + \mathrm{H.c.}) +
       k_2 \sum_{j_y=1}^{2} \sum_{j_x=1}^{N/4} (a_{j_x, j_y, \uparrow}^\dagger a_{j_x, j_y, \uparrow} - \frac{1}{2})
                                               (a_{j_x, j_y, \downarrow}^\dagger a_{j_x, j_y, \downarrow} - \frac{1}{2}) \quad (k_1=1, k_2=1)$
$a_{j_x, j_y, \uparrow} = \frac{1}{2} (X_{2((N/4)j_y+j_x)} + i Y_{2((N/4)j_y+j_x)}) \prod_{k=2((N/4)j_y+j_x)-1}^{0} Z_k, \quad
  a_{j_x, j_y, \downarrow} = \frac{1}{2} (X_{2((N/4)j_y+j_x)+1} + i Y_{2((N/4)j_y+j_x)+1}) \prod_{k=2((N/4)j_y+j_x)}^{0} Z_k$


In [4]:
def Hamiltonian_terms(n, label):
    h_terms = []
    
    if label == 0:
        h_terms.append(h_two_sites(n, "Z"))
        h_terms.append(h_single_site(n, "X"))
    if label == 1:
        h_terms.append(h_two_sites(n, "X") + h_two_sites(n, "Y") + h_two_sites(n, "Z"))
        h_terms.append(h_single_site(n, "Z"))
    if label == 2:
        h_terms.append(h_two_sites(n, "X") + h_two_sites(n, "Y") + h_two_sites(n, "Z"))
        term = qu.qarray(np.zeros((2**n, 2**n), dtype=complex))
        dims = (2,) * n
        for k in ["X", "Y", "Z"]:
            for i in range(n-1):
                term += (-1)**i * qu.ikron(qu.pauli(k), dims, [i, i+1])
        h_terms.append(term)
    if label == 3:
        h_terms.append(h_two_sites(n, "X") + h_two_sites(n, "Y") + h_two_sites(n, "Z"))
        h_terms.append(h_two_sites_j2(n, "X") + h_two_sites_j2(n, "Y") + h_two_sites_j2(n, "Z"))
    if label == 4:
        assert n % 2 == 0
        # hopping term
        hopping_term = qu.qarray(np.zeros((2**n, 2**n), dtype=complex))
        for j in range(n//2-1):
            for sigma in range(2):
                hopping_term -= creation_operator(n, 2*j+sigma) @ annihilation_operator(n, 2*(j+1)+sigma) + creation_operator(n, 2*(j+1)+sigma) @ annihilation_operator(n, 2*j+sigma)
        h_terms.append(hopping_term)
        # interaction term
        interaction_term = qu.qarray(np.zeros((2**n, 2**n), dtype=complex))
        for j in range(n//2):
            interaction_term += (creation_operator(n, 2*j) @ annihilation_operator(n, 2*j) - 1/2 * qu.identity(2**n)) @ (creation_operator(n, 2*j+1) @ annihilation_operator(n, 2*j+1) - 1/2 * qu.identity(2**n))
        h_terms.append(interaction_term)
    if label == 5:
        assert n >= 8 and n % 4 == 0
        # hopping term
        hopping_term = qu.qarray(np.zeros((2**n, 2**n), dtype=complex))
        for sigma in range(2):
            for jx in range(n//4-1):
                for jy in range(2):
                    j = n//4*jy + jx
                    hopping_term -= creation_operator(n, 2*j+sigma) @ annihilation_operator(n, 2*(j+1)+sigma) + creation_operator(n, 2*(j+1)+sigma) @ annihilation_operator(n, 2*j+sigma)
            for jx in range(n//4):
                hopping_term -= creation_operator(n, 2*jx+sigma) @ annihilation_operator(n, 2*(n//4+jx)+sigma) + creation_operator(n, 2*(n//4+jx)+sigma) @ annihilation_operator(n, 2*jx+sigma)
        h_terms.append(hopping_term)
        # interaction term
        interaction_term = qu.qarray(np.zeros((2**n, 2**n), dtype=complex))
        for jx in range(n//4):
            for jy in range(2):
                j = n//4*jy + jx
                interaction_term += (creation_operator(n, 2*j) @ annihilation_operator(n, 2*j) - 1/2 * qu.identity(2**n)) @ (creation_operator(n, 2*j+1) @ annihilation_operator(n, 2*j+1) - 1/2 * qu.identity(2**n))
        h_terms.append(interaction_term)

    return h_terms

It is defined as a function that returns a list of Hamiltonian terms so that they can be weighted by any coefficients you like.  
Note that the Hamiltonian is defined by the Pauli operator, not the spin operator.

## Sanity Check

### Label 0 (1D transversefield Ising model)

In [5]:
h_terms = Hamiltonian_terms(n, label=0)
h = h_terms[0] + 2*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-8.37679863685036
[[ 0.16887+0.j]
 [-0.21149+0.j]
 [-0.26883+0.j]
 [ 0.20489+0.j]
 [-0.26883+0.j]
 [ 0.35732+0.j]
 [ 0.26045+0.j]
 [-0.21149+0.j]
 [-0.21149+0.j]
 [ 0.26045+0.j]
 [ 0.35732+0.j]
 [-0.26883+0.j]
 [ 0.20489+0.j]
 [-0.26883+0.j]
 [-0.21149+0.j]
 [ 0.16887+0.j]]


In [6]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label0.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-8.376798636850356
[[-0.03704-0.16476j]
 [ 0.04639+0.20634j]
 [ 0.05897+0.26228j]
 [-0.04494-0.1999j ]
 [ 0.05897+0.26228j]
 [-0.07838-0.34862j]
 [-0.05713-0.2541j ]
 [ 0.04639+0.20634j]
 [ 0.04639+0.20634j]
 [-0.05713-0.2541j ]
 [-0.07838-0.34862j]
 [ 0.05897+0.26228j]
 [-0.04494-0.1999j ]
 [ 0.05897+0.26228j]
 [ 0.04639+0.20634j]
 [-0.03704-0.16476j]]


In [7]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0000000000000002

### Label 1 (1D Heisenberg model)

In [8]:
h_terms = Hamiltonian_terms(n, label=1)
h = h_terms[0] + 2*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-7.828427124746192
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.2706 +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.65328+0.j]
 [ 0.     +0.j]
 [-0.65328+0.j]
 [ 0.2706 +0.j]
 [ 0.     +0.j]]


In [9]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label1.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-7.828427124746191
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.2706 +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.65328+0.j]
 [ 0.     +0.j]
 [-0.65328+0.j]
 [ 0.2706 +0.j]
 [ 0.     +0.j]]


In [10]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0000000000000004

### Label 2 (Su-Schrieffer-Heeger model)

In [11]:
h_terms = Hamiltonian_terms(n, label=2)
h = h_terms[0] + 3/2*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-15.035653752852738
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.02375+0.j]
 [ 0.     +0.j]
 [-0.4877 +0.j]
 [ 0.51145+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.51145+0.j]
 [-0.4877 +0.j]
 [ 0.     +0.j]
 [-0.02375+0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]]


In [12]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label2.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-15.035653752852738
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.02375+0.j]
 [ 0.     +0.j]
 [ 0.4877 +0.j]
 [-0.51145+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.51145+0.j]
 [ 0.4877 +0.j]
 [-0.     +0.j]
 [ 0.02375+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]]


In [13]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0

### Label 3 (J1-J2 model)

In [14]:
h_terms = Hamiltonian_terms(n, label=3)
h = h_terms[0] + 3*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-18.16515138991169
[[ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.47034+0.j]
 [ 0.     +0.j]
 [ 0.0548 +0.j]
 [-0.52514+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.52514+0.j]
 [ 0.0548 +0.j]
 [-0.     +0.j]
 [ 0.47034+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]]


In [15]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label3.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-18.165151389911674
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.47034+0.j]
 [-0.     +0.j]
 [ 0.0548 +0.j]
 [-0.52514+0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.52514+0.j]
 [ 0.0548 +0.j]
 [-0.     +0.j]
 [ 0.47034+0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]]


In [16]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0

### Label 4 (1D Hubbard model)

In [17]:
h_terms = Hamiltonian_terms(n, label=4)
h = h_terms[0] + h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-2.0615528128088307
[[ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.43516+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.55735+0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.55735+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.43516+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]]


In [18]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label4.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-2.06155281280883
[[ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.43516+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.55735+0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.55735+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.43516+0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]]


In [19]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0000000000000002

### Label 5 (2D Hubbard model)

In [20]:
n = 8

In [21]:
h_terms = Hamiltonian_terms(n, label=5)
h = h_terms[0] + h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-4.3408476172483415
[[ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.1258 +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.16796+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.16796+0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.1258 +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.16796+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.16796+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0

In [22]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label5.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-4.340847617248348
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.1258 +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.16796+0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.16796+0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.1258 +0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [-0.16796+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [ 0.16796+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.

In [23]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0000000000000009